In [13]:
#pip install glob

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement glob (from versions: none)
ERROR: No matching distribution found for glob

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling
import glob

In [27]:
# 设定TIFF文件的目录
input_directory = 'E:/USS/CASA03_DigitalViz/GroupProject/data/temp/tiff'
output_directory = 'E:/USS/CASA03_DigitalViz/GroupProject/data/temp/tiff84'

# 用glob模块查找所有的TIFF文件
tiff_files = glob.glob(f'{input_directory}/*.tif')

# 遍历找到的TIFF文件
for file in tiff_files:
    with rasterio.open(file) as src:
        print(f"处理文件: {file}")
        print("原始坐标系统:", src.crs)
        
        # 定义新的坐标参考系统
        new_crs = 'EPSG:4326'  # WGS84，经纬度系统

        # 计算从原始坐标系统到新坐标系统的转换参数
        transform, width, height = calculate_default_transform(
            src.crs, new_crs, src.width, src.height, *src.bounds)

        # 设置新的元数据
        kwargs = src.meta.copy()
        kwargs.update({
            'crs': new_crs,
            'transform': transform,
            'width': width,
            'height': height
        })

        # 输出文件路径
        output_file = f"{output_directory}/{file.split('/')[-1]}"

        # 创建新的TIFF文件以保存转换后的数据
        with rasterio.open(output_file, 'w', **kwargs) as dst:
            for i in range(1, src.count + 1):
                # 重新投影和写入每个波段
                reproject(
                    source=rasterio.band(src, i),
                    destination=rasterio.band(dst, i),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=transform,
                    dst_crs=new_crs,
                    resampling=Resampling.nearest)
        print(f"转换完成: {output_file}")


处理文件: E:/USS/CASA03_DigitalViz/GroupProject/data/temp/tiff\A2000032.061.20.tif
原始坐标系统: GEOGCS["GCS_Unknown_datum_based_upon_the_Clarke_1866_ellipsoid",DATUM["D_Not_specified_based_on_Clarke_1866_spheroid",SPHEROID["Clarke 1866",6378206.4,294.9786982,AUTHORITY["EPSG","7008"]]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST]]
转换完成: E:/USS/CASA03_DigitalViz/GroupProject/data/temp/tiff84/tiff\A2000032.061.20.tif
处理文件: E:/USS/CASA03_DigitalViz/GroupProject/data/temp/tiff\A2000061.061.20.tif
原始坐标系统: GEOGCS["GCS_Unknown_datum_based_upon_the_Clarke_1866_ellipsoid",DATUM["D_Not_specified_based_on_Clarke_1866_spheroid",SPHEROID["Clarke 1866",6378206.4,294.9786982,AUTHORITY["EPSG","7008"]]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST]]
转换完成: E:/USS/CASA03_DigitalViz/GroupProject/data/temp/tiff84/tiff\A2000061.061.20.tif
处理文件: E:/USS/CASA0

In [2]:
import geopandas as gpd
from rasterio.mask import mask
import os

In [35]:
# 加载矢量数据
vector_data = gpd.read_file('E:/USS/CASA03_DigitalViz/GroupProject/data/city_boudaries/GBA.shp')
tiff84_data = 'E:/USS/CASA03_DigitalViz/GroupProject/data/temp/tiff_global'
output_dir = 'E:/USS/CASA03_DigitalViz/GroupProject/data/temp/tiff84'

# 用glob模块查找所有的TIFF文件
tiff84_files = glob.glob(f'{tiff84_data}/*.tif')

# 读取栅格数据
for file in tiff84_files:
    with rasterio.open(file) as src:
        # 将矢量数据转换为GeoJSON格式，这是rasterio mask操作所需的
        geoms = [feature["geometry"] for _, feature in vector_data.iterrows()]

        # 使用矢量数据裁剪栅格数据
        out_image, out_transform = mask(src, geoms, crop=True)
        
        # 更新元数据信息
        out_meta = src.meta.copy()
        out_meta.update({
            "driver": "GTiff",
            "height": out_image.shape[1],
            "width": out_image.shape[2],
            "transform": out_transform
        })

        # 保存裁剪后的栅格数据
        output_filename = os.path.splitext(os.path.basename(file))[0] + '_clipped.tif'
        output_file = f"{output_dir}/{file.split('/')[-1]}"
        
        with rasterio.open(output_file, 'w', **out_meta) as dest:
            dest.write(out_image)
        print(f"裁剪完成，输出文件保存为 '{output_dir}'")


裁剪完成，输出文件保存为 'E:/USS/CASA03_DigitalViz/GroupProject/data/temp/tiff84'
裁剪完成，输出文件保存为 'E:/USS/CASA03_DigitalViz/GroupProject/data/temp/tiff84'
裁剪完成，输出文件保存为 'E:/USS/CASA03_DigitalViz/GroupProject/data/temp/tiff84'
裁剪完成，输出文件保存为 'E:/USS/CASA03_DigitalViz/GroupProject/data/temp/tiff84'
裁剪完成，输出文件保存为 'E:/USS/CASA03_DigitalViz/GroupProject/data/temp/tiff84'
裁剪完成，输出文件保存为 'E:/USS/CASA03_DigitalViz/GroupProject/data/temp/tiff84'
裁剪完成，输出文件保存为 'E:/USS/CASA03_DigitalViz/GroupProject/data/temp/tiff84'
裁剪完成，输出文件保存为 'E:/USS/CASA03_DigitalViz/GroupProject/data/temp/tiff84'
裁剪完成，输出文件保存为 'E:/USS/CASA03_DigitalViz/GroupProject/data/temp/tiff84'
裁剪完成，输出文件保存为 'E:/USS/CASA03_DigitalViz/GroupProject/data/temp/tiff84'
裁剪完成，输出文件保存为 'E:/USS/CASA03_DigitalViz/GroupProject/data/temp/tiff84'
裁剪完成，输出文件保存为 'E:/USS/CASA03_DigitalViz/GroupProject/data/temp/tiff84'
裁剪完成，输出文件保存为 'E:/USS/CASA03_DigitalViz/GroupProject/data/temp/tiff84'
裁剪完成，输出文件保存为 'E:/USS/CASA03_DigitalViz/GroupProject/data/temp/tiff84'
裁剪完成，输出文件保存为 'E:/USS

# tiff转点shp

In [8]:
tiffGBA_data = 'E:/USS/CASA03_DigitalViz/GroupProject/data/temp/tif84/tiff_global'
output_dir = 'E:/USS/CASA03_DigitalViz/GroupProject/data/temp/tiff84'

# 用glob模块查找所有的TIFF文件
tiffGBA_files = glob.glob(f'{tiffGBA_data}/*.tif')

print("开始处理栅格文件...")

for file in tiffGBA_files:
    with rasterio.open(file) as src:
        band1 = src.read(1)  # 只关心第一波段
        transform = src.transform

        # 准备一个列表，用于存储符合条件的点
        points = []

        # 遍历栅格中的每个像素
        for row in range(band1.shape[0]):
            for col in range(band1.shape[1]):
                value = band1[row, col]
                if value > 0:  # 选择阈值，这里假设非零即有效
                    # 获得像素中心的地理坐标
                    x, y = transform * (col + 0.5, row + 0.5)
                    points.append({'geometry': Point(x, y), 'value': value})

        # 创建GeoDataFrame
        gdf_points = gpd.GeoDataFrame(points, crs=src.crs)

        # 保存裁剪后的栅格数据
        base_filename = os.path.splitext(os.path.basename(file))[0] + '.shp'
        output_file = os.path.join(output_dir, base_filename)
        
        gdf_points.to_file(output_file)

        print(f"栅格转点数据完成，输出文件已保存为 '{output_file}'")

print("所有文件处理完成。")


开始处理栅格文件...
所有文件处理完成。


# shp转geojson

In [1]:
import geopandas as gpd
import os

# 指定包含Shapefile的目录
input_directory = r"E:\USS\CASA03_DigitalViz\GroupProject\data\temp\temp_points"
output_directory = r"E:\USS\CASA03_DigitalViz\GroupProject\data\temp\temp_points\geojson"

# 创建输出目录，如果不存在的话
os.makedirs(output_directory, exist_ok=True)

# 遍历目录中的所有.shp文件
for filename in os.listdir(input_directory):
    if filename.endswith(".shp"):
        # 完整的文件路径
        input_path = os.path.join(input_directory, filename)
        # 读取Shapefile
        gdf = gpd.read_file(input_path)
        
        # 设定输出文件的路径和名称
        output_path = os.path.join(output_directory, filename.replace(".shp", ".geojson"))
        
        # 将数据以GeoJSON格式保存
        gdf.to_file(output_path, driver='GeoJSON')

print("所有Shapefile已成功转换为GeoJSON格式。")


所有Shapefile已成功转换为GeoJSON格式。
